# Lecture des échantillons 


In [2]:
# -*- coding: utf8 -*-
import pandas as pd
import pickle, glob
import itertools as it
import networkx as nx

In [33]:
debug=False
noDiff=False

In [4]:
pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
def lireLexique(nomLexique):
    with open(nomLexique, 'rb') as input:
        lexique=pickle.load(input)
    return lexique

In [6]:
listeTirages=glob.glob("/Users/gilles/Box Sync/2015-Data/*-Tirage.pkl")
listeVHF=[u"être","avoir","faire","dire","pouvoir","aller","voir","savoir","vouloir","venir"]

In [7]:
coulMF=["orange","brown","yellow","lime",
          "green","teal","lightgray","pink","cyan","magenta"]
coulMT=["blue","darkgray","purple","red","olive","violet"]
couleurCase={
    "pi2S":"orange","pi3S":"orange",
    "ii1S":"brown","ii2S":"brown","ii3S":"brown","ii3P":"brown",
    "fi1S":"yellow","fi2P":"yellow",
    "pc1S":"yellow","pc2S":"yellow","pc3S":"yellow","pc3P":"yellow",
    "fi2S":"lime","fi3S":"lime",
    "fi1P":"green","fi3P":"green",
    "ps1S":"teal","ps2S":"teal","ps3S":"teal","ps3P":"teal",
    "ai2S":"lightgray","ai3S":"lightgray","is3S":"lightgray",
    "is1S":"pink","is2S":"pink","is3P":"pink",
    "ppMS":"cyan","ppMP":"cyan",
    "ppFS":"magenta","ppFP":"magenta"
            }
listLimites=[2,8,32,128,512]
listLimCoul=["red","pink","orange","green","teal","white"]

In [8]:
def tiragesNom(nom):
    result=[tirage for tirage in listeTirages if nom in tirage]
    return result

In [9]:
def nbFormesEchantillons(nom):
    print nom
    for tirage in tiragesNom(nom):
        sample=lireLexique(tirage)
        print sample[sample["tir1"]>0]["tir1"].count(),
        print sample[sample["tir1"]>0]["tir1"].sum(),
        print tirage

In [10]:
listeTirages200=tiragesNom("200Mo")
listeTirages20=tiragesNom("20Mo")
#len(listeTirages200)
listeTirages1=tiragesNom("1Mo")
listeTirages50k=tiragesNom("50Ko")
listeTirages100k=tiragesNom("100Ko")


tailleLexique=[]
nomLexique0=listeTirages200.pop(0)
nomLexique0=listeTirages1.pop(0)
lexique=lireLexique(nomLexique0)
taille=lexique[lexique["tir1"]>0]["tir1"].count()
print taille, lexique[lexique["tir1"]>0]["tir1"].sum()
tailleLexique.append(taille)

In [11]:
def findFormNumbers(paradigme):
    dictNumbers={}
    cases=[c for c in paradigmes.columns.tolist() if c!="lexeme"]
    for case in cases:
        dictNumbers[case]=paradigme[case].count()
    return dictNumbers

In [12]:
def reduceTrad(el,traductions):
    if traductions[el]==el:
        return el
    else:
        return reduceTrad(traductions[el],traductions)
    
def defParadigme(paradigmes):
    syncretisms=[]
    cases=[c for c in paradigmes.columns.tolist() if c!="lexeme"]
    for (c1,c2) in it.combinations(cases, 2):
        c1Val=paradigmes[c1].notnull()
        c2Val=paradigmes[c2].notnull()
        c1Sur=paradigmes[c1].str.contains(",")
        c2Sur=paradigmes[c2].str.contains(",")
        l1=len(paradigmes[paradigmes[c1].notnull()])
        l2=len(paradigmes[paradigmes[c2].notnull()])
        paire=paradigmes[c1Val & c2Val & (paradigmes[c1]!=paradigmes[c2])][[c1,c2]]
        lenDiff=len(paire[~paire[c1].str.contains(",") & ~paire[c2].str.contains(",")])
        if lenDiff>0:
            if debug:
                print u"%s ≠ %s"%(c1,c2)
                print "différence",lenDiff
                if lenDiff<12:
                    print paire
        else:
            surAbondant=paire[paire[c1].str.contains(",") | paire[c2].str.contains(",")]
#            print "--------------------------------"
            if len(surAbondant)==0:
#                print u"%s = %s"%(c1,c2) 
                syncretisms.append(u"%s = %s"%(c1,c2))
            else:
                compatible=True
                for index,row in surAbondant.iterrows():
                    if "," in row[c1]:
                        if "," in row[c2]:
                            if row[c1]!=row[c2]:
                                compatible=False
                        else:
                            if not row[c2] in row[c1].split(","):
                                compatible=False
                    else:
                        if not row[c1] in row[c2].split(","):
                            compatible=False
                if compatible:
#                    print u"%s = %s"%(c1,c2)
                    syncretisms.append(u"%s = %s"%(c1,c2))
                else:
                    print u"%s ≠ %s"%(c1,c2)
                    print surAbondant

    reductionParadigme={c:c for c in cases}
    for syncretism in syncretisms:
        c1,c2=syncretism.split(" = ")
        removeC=max(c1,c2)
        keyC=min(c1,c2)
        if removeC in reductionParadigme:
            if keyC in reductionParadigme:
                key=min(reductionParadigme[keyC],reductionParadigme[removeC])
            else:
                key=min(keyC,reductionParadigme[removeC])
        else:
            if keyC in reductionParadigme:
                key=reductionParadigme[keyC]
            else:
                key=keyC
        reductionParadigme[removeC]=key
        reductionParadigme[keyC]=key

    for el in reductionParadigme:
        reductionParadigme[el]=reduceTrad(el,reductionParadigme)

    paradigmeReduit = {}
    for k, v in reductionParadigme.iteritems():
        paradigmeReduit[v] = paradigmeReduit.get(v, [])
        paradigmeReduit[v].append(k)
    print "syncrétismes",len(paradigmeReduit), sorted(paradigmeReduit.keys())
    print paradigmeReduit

In [13]:
def findClique(cliques):
    pNodes=set()

    sCliques=sorted(cliques, key=len, reverse=True)

    cliqueFound=sorted(sCliques[0])
    pNodes=pNodes|set(sCliques[0])

    removeCliques=[]
    for clique in sCliques:
        sClique=set(clique)
#        print "sClique",sClique
        if set.intersection(pNodes,sClique):
#            print "intersection",set.intersection(pNodes,sClique)
            removeCliques.append(clique)
    for clique in removeCliques:
        sCliques.remove(clique)
#    print pCliques
#    print sCliques
    return (cliqueFound,sCliques)

In [29]:
def findPartition(syncretismes):
    g=nx.Graph()
    for node in list(lexique[lexique["tir1"]>0]["case"].unique()):
        g.add_node(node)
    for syncretisme in syncretismes:
        c1,c2=syncretisme.split(" = ")
        g.add_edge(c1,c2)
    cliques.extend(list(nx.find_cliques(g)))
    sCliques=sorted(cliques, key=len, reverse=True)
    foundCliques=[]
    while sCliques:
        foundClique,sCliques=findClique(sCliques)
        foundCliques.append(foundClique)
    foundNodes=set(n for l in foundCliques for n in l )
    missingNodes=[[n] for n in g.nodes() if not n in foundNodes]
    partition=foundCliques+missingNodes
    dictPartition={l[0]:l for l in partition}
    return dictPartition

In [15]:
def drawPartition(dictPartition):
    g=nx.DiGraph()
    g.add_node("paradigme",color="red")
    for numMorphome,morphome in enumerate(dictPartition):
        g.add_node("M%02d"%numMorphome,color="blue")
        g.add_edge("paradigme","M%02d"%numMorphome)
        for case in dictPartition[morphome]:
            g.add_edge("M%02d"%numMorphome,case)
    graphs.append(g)

In [16]:
def findColours(dictPartition,debug=False):
    def assignColour(listMorphome,morphColour):
        for m in listMorphome:
            if not m in dictColours:
                dictColours[m]=morphColour
            else:
                print "conflit sur %s entre %s et %s"%(m,dictColours[m],morphColour)

    dictColours={}
    iCoul=0
    for morphome in dictPartition:
        listMorphome=dictPartition[morphome]
        lenMorphome=len(listMorphome)
        if lenMorphome>1:
            if debug: print lenMorphome,listMorphome,
            if morphome in couleurCase:
                if debug: print couleurCase[morphome]
                morphColour=couleurCase[morphome]
            else:
                noCoul=True
                for el in listMorphome:
                    if el in couleurCase:
                        if debug: print couleurCase[el]
                        morphColour=couleurCase[el]
                        noCoul=False
                        break
                if noCoul:
                    if debug: print "autre",coulMT[iCoul]
                    morphColour=coulMT[iCoul]
                    iCoul+=1
        else:
            morphColour="white"
        assignColour(listMorphome,morphColour)
    return dictColours

In [17]:
def makeTabular(dictColours,title="",coulLim=False):
    tabular=[]
    def makeLine6(tenseCode):
        line=[]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            case=tenseCode+person
            if case in dictColours:
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
            else:
                line.append(r"\cellcolor{%s}%s"%("black",case))
        return r" & ".join(line)+r"\\"

    def makeLine3(tenseCode):
        line=[]
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            if person in ["2S","1P","2P"]:
                case=tenseCode+person
                line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
            else:
                line.append(r"---")
        return r" & ".join(line)+r"\\"
    
    def makeLineNF():
        line=[]
        for case in ["inf","pP","ppMS","ppMP","ppFS","ppFP"]:
            line.append(r"\cellcolor{%s}%s"%(dictColours[case],case))
        return r" & ".join(line)+r"\\"
    
    def makeLineCoulLim():
        line=[]
        for numLimite,limite in enumerate(listLimites):
            line.append(r"\hline")
            line.append(r"\hline")
            line.append(r"\cellcolor{%s}%s"%(listLimCoul[numLimite],"$<$"+str(limite)))
        return r" & ".join(line)+r"\\"
        
    top=[
        r"\begin{center}",
        r"\begin{tabular}{cccccc}",
        r"\hline"
        ]
    bottom=[
        r"\hline",
        r"\end{tabular}\\",
        title,
        r"\end{center}",
        r"\bigskip",
        r""
        ]
    tabular.append("\n".join(top))
    for tenseCode in ["pi","ii","fi","pc", "ps","ai", "is"]:
        tabular.append(makeLine6(tenseCode))
    tabular.append(makeLine3("pI"))
    tabular.append(makeLineNF())
    if coulLim:
        tabular.append(makeLineCoulLim())
    tabular.append("\n".join(bottom))
    print "\n".join(tabular)

In [23]:
def defParadigme(paradigmes,debug=False):
#    syncretisms=[]

    cases=[c for c in paradigmes.columns.tolist() if c!="lexeme"]
    if debug: print "%d cases :"%len(cases),", ".join(cases)
    for (c1,c2) in it.combinations(cases, 2):
        c1Val=paradigmes[c1].notnull()
        c2Val=paradigmes[c2].notnull()
        c1Sur=paradigmes[c1].str.contains(",")
        c2Sur=paradigmes[c2].str.contains(",")
        l1=len(paradigmes[paradigmes[c1].notnull()])
        l2=len(paradigmes[paradigmes[c2].notnull()])
        egalPaire=paradigmes[c1Val & c2Val & (paradigmes[c1]==paradigmes[c2])][[c1,c2]]
        diffPaire=paradigmes[c1Val & c2Val & (paradigmes[c1]!=paradigmes[c2])][[c1,c2]]
        lenDiff=len(diffPaire[~diffPaire[c1].str.contains(",") & ~diffPaire[c2].str.contains(",")])
        if lenDiff>0:
            if debug:
                print u"%s ≠ %s"%(c1,c2)
                print "différence",lenDiff
                if lenDiff<12:
                    print diffPaire
        else:
            surAbondant=diffPaire[diffPaire[c1].str.contains(",") | diffPaire[c2].str.contains(",")]
#            print "--------------------------------"
#            print surAbondant
            if len(surAbondant)==0:
                if len(egalPaire)>0 or noDiff:
                    if debug: print u"%s = %s"%(c1,c2), len(egalPaire)
                    syncretisms.append(u"%s = %s"%(c1,c2))
            else:
                compatible=True
                egalCount=0
                for index,row in surAbondant.iterrows():
#                    print index,row[c1],row[c2]
                    if "," in row[c1]:
                        if "," in row[c2]:
                            setC1=set(row[c1].split(","))
                            setC2=set(row[c2].split(","))
                            print setC1&setC2
                            if not setC1&setC2:
                                compatible=False
                            else:
                                egalCount+=1
                        else:
                            if not row[c2] in row[c1].split(","):
                                compatible=False
                            else:
                                egalCount+=1
                    else:
                        if not row[c1] in row[c2].split(","):
                            compatible=False
                        else:
                            egalCount+=1
                if compatible:
                    if debug: print u"%s = %s"%(c1,c2), len(egalPaire), egalCount
                    syncretisms.append(u"%s = %s"%(c1,c2))
                else:
                    if debug: print u"%s ≠ %s"%(c1,c2)
                    if debug: print surAbondant
    dictPartition=findPartition(syncretisms)
    itemizeLines.append("%d morphomes"%len(dictPartition))
    for key in sorted(dictPartition):
        if debug: print "%s: %s," % (key, dictPartition[key]),
    if debug: print
    drawPartition(dictPartition)
    return(dictPartition)
    

In [19]:
def makeItemize(itemizeLines):
    lines=[]
    lines.append(r"\begin{itemize}")
    for line in itemizeLines:
        lines.append(r"\item "+line)
    lines.append(r"\end{itemize}")
    return "\n".join(lines)

In [20]:
def findNumberColour(dictNumbers):
    result={}
    for case in dictNumbers:
        if dictNumbers[case]<listLimites[0]:
            result[case]=listLimCoul[0]
        elif dictNumbers[case]<listLimites[1]:
            result[case]=listLimCoul[1]
        elif dictNumbers[case]<listLimites[2]:
            result[case]=listLimCoul[2]
        elif dictNumbers[case]<listLimites[3]:
            result[case]=listLimCoul[3]
        elif dictNumbers[case]<listLimites[4]:
            result[case]=listLimCoul[4]
        else:
            result[case]=listLimCoul[5]
    return result

In [34]:
graphs=[]
for nomLexique in sorted(listeTirages100k)[0:1]:
    lexique=lireLexique(nomLexique)
    taille=lexique[lexique["tir1"]>0]["tir1"].count()
    itemizeLines=[]
    print (nomLexique.split("/")[-1])
    itemizeLines.append("%d formes"%taille)
    itemizeLines.append("%d tirages"%lexique[lexique["tir1"]>0]["tir1"].sum())
    paradigmes=pd.pivot_table(lexique[lexique["tir1"]>0], values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
    dictNumbers=findFormNumbers(paradigmes)
    dictFormNumbers=findNumberColour(dictNumbers)

    syncretisms=[]
    cliques=[]

    dictPartition=defParadigme(paradigmes)
    dictColours=findColours(dictPartition)
    print makeItemize(itemizeLines)
    makeTabular(dictColours,title="Morphomes")
    makeTabular(dictFormNumbers,title="Nombre de formes",coulLim=True)

MGC-170330-00-100Ko-Tirage.pkl
\begin{itemize}
\item 2444 formes
\item 5000 tirages
\item 29 morphomes
\end{itemize}
\begin{center}
\begin{tabular}{cccccc}
\hline
\cellcolor{white}pi1S & \cellcolor{orange}pi2S & \cellcolor{orange}pi3S & \cellcolor{blue}pi1P & \cellcolor{white}pi2P & \cellcolor{white}pi3P\\
\cellcolor{brown}ii1S & \cellcolor{brown}ii2S & \cellcolor{brown}ii3S & \cellcolor{white}ii1P & \cellcolor{white}ii2P & \cellcolor{brown}ii3P\\
\cellcolor{yellow}fi1S & \cellcolor{lime}fi2S & \cellcolor{lime}fi3S & \cellcolor{white}fi1P & \cellcolor{white}fi2P & \cellcolor{white}fi3P\\
\cellcolor{yellow}pc1S & \cellcolor{yellow}pc2S & \cellcolor{yellow}pc3S & \cellcolor{white}pc1P & \cellcolor{white}pc2P & \cellcolor{yellow}pc3P\\
\cellcolor{teal}ps1S & \cellcolor{teal}ps2S & \cellcolor{teal}ps3S & \cellcolor{white}ps1P & \cellcolor{white}ps2P & \cellcolor{teal}ps3P\\
\cellcolor{white}ai1S & \cellcolor{lightgray}ai2S & \cellcolor{lightgray}ai3S & \cellcolor{white}ai1P & \cellcolor{bl

In [26]:
dictPartition

{u'ai1P': [u'ai1P', u'fi1P', u'ii2P'],
 u'ai1S': [u'ai1S'],
 u'ai2S': [u'ai2S', u'fi2P'],
 u'ai3P': [u'ai3P'],
 u'ai3S': [u'ai3S'],
 u'fi1S': [u'fi1S', u'pc1S', u'pc2S', u'pc3P', u'pc3S', u'ps1P'],
 u'fi2S': [u'fi2S'],
 u'fi3P': [u'fi3P'],
 u'fi3S': [u'fi3S'],
 u'ii1P': [u'ii1P', u'is3S', u'ppFP'],
 u'ii1S': [u'ii1S', u'ii2S', u'ii3P', u'ii3S'],
 u'inf': [u'inf'],
 u'pI1P': [u'pI1P'],
 u'pI2P': [u'pI2P'],
 u'pI2S': [u'pI2S'],
 u'pP': [u'pP'],
 u'pc1P': [u'pc1P'],
 u'pc2P': [u'pc2P'],
 u'pi1P': [u'pi1P'],
 u'pi1S': [u'pi1S'],
 u'pi2P': [u'pi2P'],
 u'pi2S': [u'pi2S'],
 u'pi3P': [u'pi3P'],
 u'pi3S': [u'pi3S'],
 u'ppFS': [u'ppFS'],
 u'ppMP': [u'ppMP'],
 u'ppMS': [u'ppMS'],
 u'ps1S': [u'ps1S', u'ps2S', u'ps3P', u'ps3S'],
 u'ps2P': [u'ps2P']}

In [35]:
sorted(cliques,key=len)

[[u'ii1P'],
 [u'pP'],
 [u'inf'],
 [u'ai3P'],
 [u'ai1P'],
 [u'pi2P'],
 [u'ps1P'],
 [u'pI2S'],
 [u'fi2P'],
 [u'pI2P'],
 [u'ps2P'],
 [u'pc2P'],
 [u'ii2P'],
 [u'pi1S'],
 [u'pc1P'],
 [u'fi3P'],
 [u'fi1P'],
 [u'ai3S', u'ai2S'],
 [u'ai3S', u'is3S'],
 [u'ps3P', u'pi3P'],
 [u'pi2S', u'pi3S'],
 [u'ai1S', u'ai2S'],
 [u'ppFS', u'ppFP'],
 [u'ppFP', u'ppMP'],
 [u'fi2S', u'fi3S'],
 [u'ai2S', u'ppMP'],
 [u'pi1P', u'pI1P'],
 [u'ppMS', u'ppMP'],
 [u'ii1S', u'ii3S', u'ii3P', u'ii2S'],
 [u'ps3P', u'ps3S', u'ps2S', u'ps1S'],
 [u'pc2S', u'pc3S', u'pc3P', u'fi1S', u'pc1S']]